In [4]:
from IPython.display import display, clear_output
import utils
import ipywidgets as widgets
from app_widgets import AppWidgets

In [ ]:
# Widgets
config = utils.read_json("config.json")

# Others

In [ ]:
# Initialise app widgets
app_widgets = AppWidgets("Miscellaneous", "config.json")

# Populate dropdown lists
app_widgets.load_dropdown_lists()

## Select experiment

In [ ]:
display(app_widgets.select_experiment_output)

with app_widgets.select_experiment_output:
    display(app_widgets.experiments_dropdown_boxes)

app_widgets.create_button.on_click(app_widgets.create_miscellaneous_action)
app_widgets.quit_button.on_click(app_widgets.close_notebook)

display(utils.Markdown(data = "### Properties"))
display(app_widgets.property_widgets["$name"], app_widgets.property_widgets["comments"])

display(utils.Markdown(data = "### Support Files"))
display(app_widgets.support_files_uploader)

display(utils.Markdown(data = "### Save"))
app_widgets.property_widgets["$name"].observe(app_widgets.update_text, names = 'value')
display(app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)